In [1]:
# Imports the Google Cloud client library

import bq_helper
from bq_helper import BigQueryHelper

import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = str('optical-aviary-293419-f44a1284c5a5.json')


from plotly import __version__

from plotly.offline import init_notebook_mode, iplot

import plotly.graph_objs as go
import matplotlib.pyplot as plt
import chart_studio.plotly as py
from plotly import tools


import pandas as pd
init_notebook_mode(connected=True)

# https://github.com/SohierDane/BigQuery_Helper
from bq_helper import BigQueryHelper
medicare = bq_helper.BigQueryHelper(active_project="bigquery-public-data", dataset_name="cms_medicare")
bq_assistant = BigQueryHelper("bigquery-public-data", "cms_medicare")

bq_assistant.list_tables()





['home_health_agencies_2013',
 'home_health_agencies_2014',
 'hospice_providers_2014',
 'hospital_general_info',
 'inpatient_charges_2011',
 'inpatient_charges_2012',
 'inpatient_charges_2013',
 'inpatient_charges_2014',
 'inpatient_charges_2015',
 'nursing_facilities_2013',
 'nursing_facilities_2014',
 'outpatient_charges_2011',
 'outpatient_charges_2012',
 'outpatient_charges_2013',
 'outpatient_charges_2014',
 'outpatient_charges_2015',
 'part_d_prescriber_2014',
 'physicians_and_other_supplier_2012',
 'physicians_and_other_supplier_2013',
 'physicians_and_other_supplier_2014',
 'physicians_and_other_supplier_2015',
 'referring_durable_medical_equip_2013',
 'referring_durable_medical_equip_2014']

In [2]:
query1 = """SELECT 

nppes_provider_state AS state,
ROUND(SUM(total_claim_count) / 1e6) AS total_claim_count_millions
FROM
  `bigquery-public-data.medicare.part_d_prescriber_2014`
GROUP BY
  state
ORDER BY
  total_claim_count_millions DESC;"""
total_count_query = medicare.query_to_pandas_safe(query1)
total_count_query.head(10)

,state,total_claim_count_millions
0,CA,116.0
1,FL,91.0
2,NY,80.0
3,TX,76.0
4,PA,63.0
5,OH,53.0
6,NC,46.0
7,IL,43.0
8,MI,39.0
9,GA,39.0


In [3]:
#scl = [[0.0, 'rgb(248,255,206)'],[0.2, 'rgb(203,255,205)'],[0.4, 'rgb(155,255,164)'], [0.6, 'rgb(79,255,178)'],[0.8, 'rgb(15,183,132)'], [1, '#008059']]
scl = [[0.0, 'rgb(255,248,206)'],[0.2, 'rgb(255,203,205)'],[0.4, 'rgb(255,155,164)'], [0.6, 'rgb(255,79,178)'],[0.8, 'rgb(183,15,132)'], [1, '#008059']]
data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = total_count_query.state,
        z = total_count_query.total_claim_count_millions,
        locationmode = 'USA-states',
        text = total_count_query.state,
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Total Claims in Different States")
        )
       ]

layout = dict(
        title = 'Total Claims in Different States',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )
iplot( fig, filename='d3-cloropleth-map' )

In [4]:
query2 = """SELECT
drug_name AS drug,
  ROUND(SUM(total_drug_cost) / 1e6) AS total_drug_count_millions
FROM
  `bigquery-public-data.medicare.part_d_prescriber_2014`
GROUP BY
  drug
ORDER BY
  total_drug_count_millions DESC;"""
total_drug_query = medicare.query_to_pandas_safe(query2)
total_drug_query.head(10)

,drug,total_drug_count_millions
0,SOVALDI,2486.0
1,NEXIUM,2422.0
2,CRESTOR,2396.0
3,ABILIFY,2247.0
4,ADVAIR DISKUS,2065.0
5,SPIRIVA,1957.0
6,LANTUS SOLOSTAR,1806.0
7,JANUVIA,1592.0
8,LANTUS,1512.0
9,REVLIMID,1501.0


In [5]:
tempdf = total_drug_query.head(30)
y = list(reversed(list(tempdf.drug)))
x = list(reversed(list(tempdf.total_drug_count_millions)))
trace1 = go.Bar(x=x, y=y, orientation="h", marker=dict(color='#f79284'),)
layout = dict(height=400, width=700, margin=dict(l=400), title='Drug Ranking per total cost')
data = [trace1]

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='stacked-bar')

In [6]:
query1 = """SELECT
  COUNT(DISTINCT npi) AS npi_id,
  nppes_provider_state AS state,
  ROUND(SUM(total_claim_count) / 1e6) AS total_claim_count_millions,
  ROUND(SUM(total_day_supply) / 1e6) AS total_day_supply_millions,
  ROUND(SUM(total_drug_cost) / 1e6) AS total_drug_cost_millions
FROM
  `bigquery-public-data.cms_medicare.part_d_prescriber_2014`
GROUP BY
  state
ORDER BY
  total_claim_count_millions DESC;
        """
total_state__claim_drug_query = medicare.query_to_pandas_safe(query1)
total_state__claim_drug_query.head(10)



,npi_id,state,total_claim_count_millions,total_day_supply_millions,total_drug_cost_millions
0,86586,CA,116.0,4935.0,9633.0
1,49874,FL,91.0,3769.0,6970.0
2,63751,NY,80.0,3150.0,7522.0
3,50853,TX,76.0,3105.0,6462.0
4,41440,PA,63.0,2402.0,4842.0
5,33064,OH,53.0,2181.0,4051.0
6,26875,NC,46.0,1706.0,3358.0
7,32415,IL,43.0,1724.0,3121.0
8,29898,MI,39.0,1797.0,3193.0
9,20728,GA,39.0,1403.0,2787.0


In [7]:
total_state__claim_drug_query.count

<bound method DataFrame.count of     npi_id state  total_claim_count_millions  total_day_supply_millions  \
0    86586    CA                       116.0                     4935.0   
1    49874    FL                        91.0                     3769.0   
2    63751    NY                        80.0                     3150.0   
3    50853    TX                        76.0                     3105.0   
4    41440    PA                        63.0                     2402.0   
..     ...   ...                         ...                        ...   
56       2    AS                         0.0                        0.0   
57       5    XX                         0.0                        0.0   
58      32    AE                         0.0                        0.0   
59       7    AA                         0.0                        0.0   
60      16    MP                         0.0                        0.0   

    total_drug_cost_millions  
0                     9633.0  
1   

In [8]:
trace1 = go.Scatter(
    x=total_state__claim_drug_query.total_day_supply_millions,
    y=total_state__claim_drug_query.total_drug_cost_millions,
    mode='markers',
    text=total_state__claim_drug_query.state,
    marker=dict(
        color=list(reversed([i*11 for i in range(200,251)])),
        size=total_state__claim_drug_query.total_claim_count_millions,
        line=dict(
            width=2
        ),
    )
)
data = [trace1]

layout = go.Layout(
    title='Total claim count | Total Drug Cost | Total Day Supply for every state',
    xaxis=dict(
        title='Total Day Supply',
        gridcolor='rgb(255, 255, 255)',
        zerolinewidth=1,
        ticklen=5,
        gridwidth=2,
    ),
    yaxis=dict(
        title='Total Drug Cost',
        gridcolor='rgb(255, 255, 255)',
        zerolinewidth=1,
        ticklen=5,
        gridwidth=2,
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='scatter-colorscale')


In [9]:
query1 = """SELECT
 # COUNT(DISTINCT npi) AS NPI,
  generic_name AS drug_name,
  ROUND(SUM(total_claim_count) / 1e6) AS total_claim_count_millions,
  ROUND(SUM(total_day_supply) / 1e6) AS total_day_supply_millions,
  ROUND(SUM(total_drug_cost) / 1e6) AS total_drug_cost_millions,
  MAX(total_claim_count) AS max_claim,
  MAX(total_day_supply)  AS max_day,
  MAX(total_drug_cost) AS  max_cost,
  VARIANCE(total_claim_count) AS var_claim,
  VARIANCE(total_day_supply) AS var_day,
  VARIANCE(total_drug_cost) AS var_cost

FROM
  `bigquery-public-data.cms_medicare.part_d_prescriber_2014`
GROUP BY
  drug_name
ORDER BY
  total_claim_count_millions DESC;
        """
most_pres_drug = medicare.query_to_pandas_safe(query1)
most_pres_drug.head(10)


,drug_name,total_claim_count_millions,total_day_supply_millions,total_drug_cost_millions,max_claim,max_day,max_cost,var_claim,var_day,var_cost
0,LEVOTHYROXINE SODIUM,42.0,2153.0,831.0,6498,171754,152571.44,31942.245714,7.133009e+07,1.183943e+07
1,LISINOPRIL,38.0,1954.0,278.0,5923,129872,45180.97,26393.958291,6.686735e+07,1.564915e+06
2,AMLODIPINE BESYLATE,36.0,1774.0,302.0,7516,141792,62784.00,29720.516914,6.275274e+07,2.100584e+06
3,SIMVASTATIN,34.0,1844.0,343.0,3994,165190,53232.74,28001.051889,7.901854e+07,3.092799e+06
4,ATORVASTATIN CALCIUM,32.0,1689.0,748.0,4687,188889,132071.09,24893.911274,6.696100e+07,1.459193e+07
5,OMEPRAZOLE,32.0,1487.0,519.0,4884,142942,78243.60,24583.781444,4.729918e+07,6.644056e+06
6,HYDROCODONE/ACETAMINOPHEN,32.0,656.0,669.0,7979,237610,253168.34,34546.738889,2.440737e+07,2.095438e+07
7,METFORMIN HCL,27.0,1355.0,326.0,2670,98912,116479.46,12609.643749,2.853354e+07,2.984249e+06
8,FUROSEMIDE,26.0,1077.0,133.0,8070,183059,49104.29,27231.562637,2.852059e+07,8.094267e+05
9,GABAPENTIN,21.0,820.0,491.0,5255,136212,140668.81,13591.728659,1.678917e+07,7.766494e+06


In [10]:
tempdf = most_pres_drug.head(50)

trace1 = go.Bar(
    x=tempdf.drug_name,
    y=tempdf.total_claim_count_millions,
    name='Total Claims'
)
trace2 = go.Bar(
    x=tempdf.drug_name,
    y=tempdf.total_day_supply_millions,
    name='Total Day Supply'
)
trace3 = go.Bar(
    x=tempdf.drug_name,
    y=tempdf.total_drug_cost_millions,
    name='Total Drug Cost'
)

data = [trace1, trace2, trace3]
layout = go.Layout(
    barmode='group'
    , margin=dict(b=200)
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='grouped-bar')

In [11]:
query1 = """SELECT
  
  specialty_description AS specialty_description,
  ROUND(SUM(total_claim_count) / 1e6) AS total_claim_count_millions,
  ROUND(SUM(total_day_supply) / 1e6) AS total_day_supply_millions,
  ROUND(SUM(total_drug_cost) / 1e6) AS total_drug_cost_millions
FROM
  `bigquery-public-data.cms_medicare.part_d_prescriber_2014`
GROUP BY
  specialty_description
ORDER BY
  total_claim_count_millions DESC;
        """
specialty_query = medicare.query_to_pandas_safe(query1)
specialty_query.head(40)

,specialty_description,total_claim_count_millions,total_day_supply_millions,total_drug_cost_millions
0,Internal Medicine,386.0,16340.0,22983.0
1,Family Practice,361.0,14967.0,18254.0
2,Nurse Practitioner,78.0,2873.0,6402.0
3,Cardiology,67.0,3401.0,4039.0
4,Psychiatry,41.0,1307.0,4191.0
5,Physician Assistant,40.0,1493.0,3020.0
6,General Practice,30.0,1046.0,1425.0
7,Ophthalmology,24.0,816.0,2135.0
8,Neurology,19.0,696.0,5087.0
9,Nephrology,14.0,575.0,1825.0


In [12]:
tempdf = specialty_query.head(40)
trace1 = go.Bar(
    x=tempdf.specialty_description,
    y=tempdf.total_claim_count_millions,
    name='Total Claims'
)
trace2 = go.Bar(
    x=tempdf.specialty_description,
    y=tempdf.total_day_supply_millions,
    name='Total Day Supply'
)
trace3 = go.Bar(
    x=tempdf.specialty_description,
    y=tempdf.total_drug_cost_millions,
    name='Total Drug Cost'
)

data = [trace1, trace2, trace3]
layout = go.Layout(
    barmode='stack'
    , margin=dict(b=200)
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='grouped-bar')



In [13]:
query1 = """SELECT 
A.state, B.drug_name, A.MaxClaimCount from (select state, MAX(total_claim_count_millions) as MaxClaimCount 
from (
SELECT
  nppes_provider_state AS state,
  generic_name AS drug_name,
  SUM(total_claim_count) AS total_claim_count_millions
FROM
  `bigquery-public-data.cms_medicare.part_d_prescriber_2014`
GROUP BY
  state, drug_name
ORDER BY
  total_claim_count_millions DESC) group by state) A INNER JOIN (SELECT
  nppes_provider_state AS state,
  generic_name AS drug_name,
  SUM(total_claim_count) AS total_claim_count_millions
FROM
  `bigquery-public-data.cms_medicare.part_d_prescriber_2014`
GROUP BY
  state, drug_name
ORDER BY
  total_claim_count_millions DESC) B ON A.MaxClaimCount = B.total_claim_count_millions;
"""
state_most_drug = medicare.query_to_pandas_safe(query1)
state_most_drug.head(10)

,state,drug_name,MaxClaimCount
0,OR,LEVOTHYROXINE SODIUM,571237
1,MN,LISINOPRIL,692905
2,MT,LEVOTHYROXINE SODIUM,155249
3,LA,HYDROCODONE/ACETAMINOPHEN,795932
4,VA,LEVOTHYROXINE SODIUM,837399
5,OK,HYDROCODONE/ACETAMINOPHEN,638873
6,WA,LISINOPRIL,771435
7,UT,LEVOTHYROXINE SODIUM,253237
8,TX,HYDROCODONE/ACETAMINOPHEN,2833795
9,CA,LEVOTHYROXINE SODIUM,3844722


In [14]:
visited = {}
rows = []
for index, row in state_most_drug.iterrows():
    stat = row['state']
    if stat not in visited:
        visited[stat] = 1
        rows.append(row)
newDF = pd.DataFrame(rows).reset_index()
newDF[['state', 'drug_name', 'MaxClaimCount']].head(41)

,state,drug_name,MaxClaimCount
0,OR,LEVOTHYROXINE SODIUM,571237
1,MN,LISINOPRIL,692905
2,MT,LEVOTHYROXINE SODIUM,155249
3,LA,HYDROCODONE/ACETAMINOPHEN,795932
4,VA,LEVOTHYROXINE SODIUM,837399
5,OK,HYDROCODONE/ACETAMINOPHEN,638873
6,WA,LISINOPRIL,771435
7,UT,LEVOTHYROXINE SODIUM,253237
8,TX,HYDROCODONE/ACETAMINOPHEN,2833795
9,CA,LEVOTHYROXINE SODIUM,3844722


In [15]:
a = newDF['drug_name'].value_counts()
labels = a.index
values = a.values
colors = ['lightblue','gray','#eee','#999', '#9f9f']
trace = go.Pie(labels=labels, values=values, hoverinfo='label+percent', 
               textinfo='value', name='Top Drugs Used',
               marker=dict(colors=colors))
layout = dict(title = 'Top Drugs in Each State',
              xaxis= dict(title= 'Drug Name', ticklen= 5,zeroline= False),
              width=800
             )
fig = dict(data = [trace], layout = layout)
iplot(fig)